In [4]:
# ### Mount Drive

# from google.colab import drive
# drive.mount('/content/drive/')

In [7]:
### Imports

import os, requests, json
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from sklearn import preprocessing
import sys

In [173]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAGXdTwEAAAAAr2%2BC9Wi6GHR8%2Bk%2FiDL2AIHaC1I8%3D86fg9nIXAt2MFp0QP1sXU0q1VFKHAGaD1da68qG4X0glvGSh4D"

def response_health(r):
  if r.status_code != 200:
    raise Exception(
    "Request returned an error: {} {}".format(
      r.status_code, r.text
    )
  )
    
def bearer_oauth(r):
  r.headers["Authorization"] = f"Bearer {bearer_token}"
  return r

def send_request(url, params=None):
  '''Send Request (url) with optional params. Returns json'''
  # https://2.python-requests.org/en/master/api/#requests.request
  if params == None:
    response = requests.request("GET", url, auth=bearer_oauth)
  else:
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
  print("Request response status: ", response.status_code)
  response_health(response)
  return response.json()

In [172]:
def get_user_id(name):
  user_json = send_request(f"https://api.twitter.com/2/users/by/username/{name}",params={"user.fields":"id"})
  id = user_json["data"]["id"]
  return id

In [171]:
def get_tweets_user(id):
    expansions = {"expansions":"author_id, attachments.poll_ids, attachments.media_keys, entities.mentions.username, geo.place_id, in_reply_to_user_id, referenced_tweets.id,referenced_tweets.id.author_id".replace(" ", "")}
    tweetFields = {"tweet.fields":"attachments, author_id, context_annotations, conversation_id, created_at, entities, geo, id, in_reply_to_user_id, lang, public_metrics, possibly_sensitive, referenced_tweets, reply_settings, source, text, withheld".replace(" ", "")}
    userFields = {"user.fields":"created_at, description, entities, id, location, name, pinned_tweet_id, profile_image_url, protected, public_metrics, url, username, verified, withheld".replace(" ", "")}
    
    # join the dictionaries
    params = {**expansions,**userFields,**tweetFields}
    tweet_json = send_request(f"https://api.twitter.com/2/users/{id}/tweets", params=params)
    return tweet_json

In [146]:
userId = get_user_id("McDonalds")
tweetJson = get_tweets_user(userId)

Request response status:  200
Request response status:  200


In [174]:
tweetJson

{'data': [{'text': '@momhatstudios Quality is of utmost importance to us! Share more info here: https://t.co/WhvwT8dw7y',
   'created_at': '2021-12-11T18:39:28.000Z',
   'entities': {'mentions': [{'start': 0,
      'end': 14,
      'username': 'momhatstudios',
      'id': '821517546088067076'}],
    'urls': [{'start': 76,
      'end': 99,
      'url': 'https://t.co/WhvwT8dw7y',
      'expanded_url': 'https://www.mcdonalds.com/us/en-us/contact-us/restaurant-feedback.html/twitter',
      'display_url': 'mcdonalds.com/us/en-us/conta…',
      'images': [{'url': 'https://pbs.twimg.com/news_img/1468632616727461905/sMPZX5h7?format=jpg&name=orig',
        'width': 108,
        'height': 108},
       {'url': 'https://pbs.twimg.com/news_img/1468632616727461905/sMPZX5h7?format=jpg&name=150x150',
        'width': 108,
        'height': 108}],
      'status': 200,
      'title': 'U.S. Restaurant Feedback: Leave a Compliment or Complaint | McDonald’s',
      'description': 'U.S. Restaurant Feedback:

In [175]:
# When it's ready, I imagine we would put this inside the get_tweets_user function
userData = tweetJson['includes']['users'][0]
tweetData = tweetJson['data']

outputDict = {"userData": {}, "tweetData": []}


outputDict["userData"]['following_count'] = userData['public_metrics']['following_count']
outputDict["userData"]['tweet_count'] = userData['public_metrics']['tweet_count']
outputDict["userData"]['followers_count'] = userData['public_metrics']['followers_count']
outputDict["userData"]['listed_count'] = userData['public_metrics']['listed_count']
outputDict["userData"]['username'] = userData['username']
outputDict["userData"]['name'] = userData['name']
outputDict["userData"]['id'] = userData['id']
outputDict["userData"]['verified'] = userData['verified']
outputDict["userData"]['protected'] = userData['protected']
outputDict["userData"]['created_at'] = userData['created_at']
outputDict["userData"]['description'] = userData['description']
outputDict["userData"]['id'] = userData['id']
outputDict["userData"]['name'] = userData['name']
outputDict["userData"]['pinned_tweet_id'] = userData['pinned_tweet_id']


for tweet in tweetData:
    tweetDict = {}
    tweetDict['text'] = tweet['text']
    tweetDict['lang'] = tweet['lang']
    tweetDict['possibly_sensitive'] = tweet['possibly_sensitive']
    tweetDict['retweet_count'] = tweet['public_metrics']['retweet_count']
    tweetDict['reply_count'] = tweet['public_metrics']['reply_count']
    tweetDict['like_count'] = tweet['public_metrics']['like_count']
    tweetDict['quote_count'] = tweet['public_metrics']['quote_count']
    tweetDict['reply_settings'] = tweet['reply_settings']
    tweetDict['num_referenced_tweets'] = len(tweet['referenced_tweets'])
    tweetDict['source']= tweet['source']
    tweetDict['created_at'] = tweet['created_at']

    #just grabbing the first url
    try:
        tweetDict['url_title'] = tweet['entities']['urls'][0]['title']
    except:
        tweetDict['url_title'] = ""
    try:
        tweetDict['url_image'] = tweet['entities']['urls'][0]['images'][0]['url']#just grabbing the first image in the first url
    except:
        tweetDict['url_image'] = ""  

    # tweetDict['text'] = tweet['text']
    # tweetDict['text'] = tweet['text']
    # tweetDict['text'] = tweet['text']
    # tweetDict['text'] = tweet['text']


    outputDict["tweetData"].append(tweetDict)

outputDict
# tweetJson


{'userData': {'following_count': 13543,
  'tweet_count': 777029,
  'followers_count': 4393793,
  'listed_count': 9061,
  'username': 'McDonalds',
  'name': "McDonald's",
  'id': '71026122',
  'verified': True,
  'protected': False,
  'created_at': '2009-09-02T17:46:43.000Z',
  'description': 'grimace is a close personal friend of mine',
  'pinned_tweet_id': '1469332042232049665'},
 'tweetData': [{'text': '@momhatstudios Quality is of utmost importance to us! Share more info here: https://t.co/WhvwT8dw7y',
   'lang': 'en',
   'possibly_sensitive': False,
   'retweet_count': 0,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0,
   'reply_settings': 'everyone',
   'num_referenced_tweets': 1,
   'source': 'Sprinklr',
   'created_at': '2021-12-11T18:39:28.000Z',
   'url_title': 'U.S. Restaurant Feedback: Leave a Compliment or Complaint | McDonald’s',
   'url_image': 'https://pbs.twimg.com/news_img/1468632616727461905/sMPZX5h7?format=jpg&name=orig'},
  {'text': '@LadyBobbi1 Mind t

In [117]:
expansions = {"expansions":"author_id, attachments.poll_ids, attachments.media_keys, entities.mentions.username, geo.place_id, in_reply_to_user_id, referenced_tweets.id,referenced_tweets.id.author_id".replace(" ", "")}
tweetFields = {"tweet.fields":"attachments, author_id, context_annotations, conversation_id, created_at, entities, geo, id, in_reply_to_user_id, lang, non_public_metrics, public_metrics, organic_metrics, promoted_metrics, possibly_sensitive, referenced_tweets, reply_settings, source, text, withheld".replace(" ", "")}
userFields = {"user.fields":"created_at, description, entities, id, location, name, pinned_tweet_id, profile_image_url, protected, public_metrics, url, username, verified, withheld".replace(" ", "")}


print({**expansions,**userFields,**tweetFields})

{'expansions': 'author_id,attachments.poll_ids,attachments.media_keys,entities.mentions.username,geo.place_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id', 'user.fields': 'created_at,description,entities,id,location,name,pinned_tweet_id,profile_image_url,protected,public_metrics,url,username,verified,withheld', 'tweet.fields': 'attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,non_public_metrics,public_metrics,organic_metrics,promoted_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld'}
